# 論文閱讀助手 - Paper Reading Assistant




## 專案簡介

這是一個基於 OpenAI GPT-5 Response API 的論文閱讀助手，幫助學生更好地理解學術論文。

**核心特色**：
- 📄 支援 PDF 上傳，完整解析論文內容
- 💬 多輪對話，可以持續提問
- 🧠 費曼學習法：用簡單語言解釋複雜概念
- 🎯 蘇格拉底式提問：引導深度思考
- 🔄 可清除對話，重新開始

---

## 1. 安裝必要套件

In [ ]:
!pip install openai gradio pypdf2


---

## 2. API Key 設定

請在 Colab 左側的 🔑 Secrets 中設定您的 OpenAI API Key，名稱為 `OpenAI`。

In [ ]:
from google.colab import userdata
import os

api_key = userdata.get('OpenAI')
os.environ['OPENAI_API_KEY'] = api_key


---

## 3. 匯入必要套件

In [ ]:
from openai import OpenAI
import gradio as gr
import PyPDF2
import io


---

## 4. 初始化 OpenAI Client

In [ ]:
client = OpenAI()
model = "gpt-5"


---

## 5. PDF 文字提取函數

In [ ]:
def extract_pdf_text(pdf_file):
    """
    從上傳的 PDF 檔案中提取文字內容

    Args:
        pdf_file: Gradio 上傳的檔案物件

    Returns:
        str: 提取的文字內容
    """
    if pdf_file is None:
        return None

    try:
        # 讀取 PDF 檔案
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # 提取所有頁面的文字
        text = ""
        for page_num, page in enumerate(pdf_reader.pages):
            page_text = page.extract_text()
            text += f"\n--- Page {page_num + 1} ---\n{page_text}\n"

        return text

    except Exception as e:
        return f"❌ PDF 讀取失敗：{str(e)}"


---

## 6. System Prompt 設定

這是論文閱讀助手的「人設」，定義了它如何幫助學生理解論文。

In [ ]:
SYSTEM_PROMPT = """你是一位專業的論文閱讀助手，專門幫助學生理解學術論文。

**你的教學原則**：

1. **費曼學習法 (Feynman Technique)**：
   - 用最簡單的語言解釋複雜概念
   - 使用類比和日常生活的例子
   - 避免過度使用專業術語，必要時要先解釋

2. **蘇格拉底式提問 (Socratic Method)**：
   - 不直接給答案，而是引導學生思考
   - 提出啟發性問題，幫助學生自己找到答案
   - 鼓勵批判性思維

3. **結構化分析**：
   - 幫助學生理解論文結構：摘要、引言、方法、結果、結論
   - 指出論文的核心貢獻和創新點
   - 解釋研究方法和實驗設計

4. **友善互動**：
   - 以鼓勵和支持的語氣回應
   - 確認學生理解後再繼續
   - 可以用 emoji 讓對話更生動

**當學生上傳論文後**：
- 等待學生提問，不要主動摘要
- 根據學生的問題，從論文中找到相關內容回答
- 確保回答準確且基於論文內容

**記住**：你的目標是幫助學生「學會如何讀論文」，而不只是「讀懂這篇論文」。"""

WELCOME_MESSAGE = """👋 嗨！我是你的**論文閱讀助手**！

📚 **我能幫你做什麼？**
- 用簡單的語言解釋論文中的複雜概念
- 幫你理解研究方法和實驗設計
- 引導你思考論文的核心貢獻
- 回答你對論文內容的任何疑問

🚀 **如何開始？**
1. 點擊上方「上傳 PDF」按鈕，選擇你要閱讀的論文
2. 上傳完成後，開始向我提問吧！

💡 **提問範例**：
- "這篇論文的主要貢獻是什麼？"
- "能用簡單的話解釋這個方法嗎？"
- "實驗結果說明了什麼？"
- "這個公式是什麼意思？"

準備好了嗎？上傳你的論文，讓我們開始吧！ 📖✨"""


---

## 7. 對話狀態管理

使用全域變數管理 PDF 內容和對話歷史。

In [ ]:
# 全域狀態
pdf_content = None
conversation_history = []


---

## 8. 核心對話函數

In [ ]:
def chat_with_paper(message, history):
    """
    處理使用者訊息並產生回應

    Args:
        message: 使用者當前輸入
        history: Gradio 聊天歷史 (list of [user_msg, bot_msg])

    Returns:
        str: 助手的回應
    """
    global pdf_content, conversation_history

    # 檢查是否已上傳 PDF
    if pdf_content is None:
        return "⚠️ 請先上傳 PDF 論文檔案！點擊上方「上傳 PDF」按鈕。"

    if pdf_content.startswith("❌"):
        return pdf_content  # 回傳錯誤訊息

    try:
        # 建構訊息列表
        messages = [
            {"role": "developer", "content": SYSTEM_PROMPT}
        ]

        # 第一次提問時，包含 PDF 內容
        if len(conversation_history) == 0:
            messages.append({
                "role": "user",
                "content": f"以下是我要閱讀的論文內容：\n\n{pdf_content}\n\n---\n\n現在我的問題是：{message}"
            })
        else:
            # 後續對話，加入歷史訊息
            messages.extend(conversation_history)
            messages.append({
                "role": "user",
                "content": message
            })

        # 呼叫 OpenAI Response API
        response = client.responses.create(
            model=model,
            input=messages,
            reasoning={"effort": "medium"},
            text={"verbosity": "medium"}
        )

        # 取得回應
        reply = response.output_text

        # 更新對話歷史（包含完整的 output）
        conversation_history.extend(response.output)

        # 更新 Gradio 顯示的歷史（重要！必須回傳完整 history）
        history = history + [[message, reply]]

        return history

    except Exception as e:
        return f"❌ 發生錯誤：{str(e)}\n\n請檢查您的 API Key 是否正確設定。"


def upload_pdf(pdf_file):
    """
    處理 PDF 上傳

    Args:
        pdf_file: Gradio 上傳的檔案

    Returns:
        str: 上傳狀態訊息
    """
    global pdf_content, conversation_history

    if pdf_file is None:
        return "❌ 請選擇 PDF 檔案"

    # 提取 PDF 文字
    pdf_content = extract_pdf_text(pdf_file)

    # 重置對話歷史
    conversation_history = []

    if pdf_content and not pdf_content.startswith("❌"):
        # 計算字數
        char_count = len(pdf_content)
        page_count = pdf_content.count("--- Page")

        return f"✅ PDF 上傳成功！\n\n📄 共 {page_count} 頁，約 {char_count:,} 字元\n\n💬 現在可以開始向我提問了！"
    else:
        return pdf_content  # 回傳錯誤訊息


def clear_conversation():
    """
    清除對話歷史，重新開始

    Returns:
        tuple: (清空的聊天歷史, 狀態訊息)
    """
    global conversation_history
    conversation_history = []

    return [], "🔄 對話已清除！你可以重新提問，或上傳新的 PDF。"


---

## 9. Gradio 介面設計

In [ ]:
# 建立 Gradio 介面
with gr.Blocks(title="論文閱讀助手", theme=gr.themes.Soft()) as demo:

    gr.Markdown("# 📚 論文閱讀助手 - Paper Reading Assistant")
    gr.Markdown("基於 OpenAI GPT-5 Response API，結合費曼學習法與蘇格拉底式提問")

    with gr.Row():
        with gr.Column(scale=3):
            # PDF 上傳區
            pdf_upload = gr.File(
                label="📄 上傳論文 PDF",
                file_types=[".pdf"],
                type="filepath"
            )
            upload_status = gr.Textbox(
                label="上傳狀態",
                value=WELCOME_MESSAGE,
                interactive=False,
                lines=8
            )

        with gr.Column(scale=7):
            # 聊天區
            chatbot = gr.Chatbot(
                label="💬 對話區",
                height=500,
                show_label=True,
                type="tuples"  # 明確指定使用 tuples 格式
            )

            msg_input = gr.Textbox(
                label="輸入你的問題",
                placeholder="例如：這篇論文的主要貢獻是什麼？",
                lines=2
            )

            with gr.Row():
                submit_btn = gr.Button("📤 送出", variant="primary")
                clear_btn = gr.Button("🔄 清除對話")

    # 事件綁定
    pdf_upload.change(
        fn=upload_pdf,
        inputs=pdf_upload,
        outputs=upload_status
    )

    submit_btn.click(
        fn=chat_with_paper,
        inputs=[msg_input, chatbot],
        outputs=chatbot
    ).then(
        lambda: "",  # 清空輸入框
        outputs=msg_input
    )

    msg_input.submit(
        fn=chat_with_paper,
        inputs=[msg_input, chatbot],
        outputs=chatbot
    ).then(
        lambda: "",  # 清空輸入框
        outputs=msg_input
    )

    clear_btn.click(
        fn=clear_conversation,
        outputs=[chatbot, upload_status]
    )

    # 說明區
    gr.Markdown("""
    ---
    ### 💡 使用技巧

    - **第一次提問**：建議先問「這篇論文在研究什麼？」了解全貌
    - **深入理解**：針對不懂的章節或概念提問
    - **批判思考**：可以問「這個方法有什麼限制？」
    - **清除對話**：想重新開始時，點擊「清除對話」按鈕

    ### ⚙️ 技術說明

    - **模型**：OpenAI GPT-5 (Response API)
    - **推理等級**：Medium (平衡速度與品質)
    - **PDF 處理**：PyPDF2 (完整文字提取)
    - **介面框架**：Gradio 5.x

    ---
    *Made with ❤️ for NCCU AI Course*
    """)


---

## 10. 啟動應用

In [ ]:
# 啟動 Gradio 應用
demo.launch(share=True, debug=True)


**啟動後**：
- Gradio 會產生一個公開連結（例如：`https://xxx.gradio.live`）
- 這個連結可以分享給任何人使用
- 連結有效期：72 小時

---

## 📸 使用範例截圖

*(在實際使用時，記得截圖以下畫面)*

1. **上傳 PDF 成功畫面**
2. **第一次提問 + 回應**
3. **多輪對話展示**
4. **費曼學習法解釋範例**
5. **蘇格拉底式提問範例**

---

## 🎓 學習重點

### Response API vs Chat Completions

這個專案使用了最新的 **OpenAI Response API**，相較於舊版 Chat Completions API：

In [ ]:
# ❌ 舊版 Chat Completions API
response = client.chat.completions.create(
    model="gpt-4",
    messages=[...]
)
reply = response.choices[0].message.content

# ✅ 新版 Response API
response = client.responses.create(
    model="gpt-5",
    input=[...],
    reasoning={"effort": "medium"},
    text={"verbosity": "medium"}
)
reply = response.output_text


**主要優勢**：
- 支援推理等級控制（`reasoning`）
- 支援輸出詳細度控制（`text.verbosity`）
- 更好的對話歷史管理（`response.output`）
- 專為 GPT-5 等推理模型優化

### Gradio 狀態管理

使用全域變數管理應用狀態：
- `pdf_content`：儲存完整 PDF 文字
- `conversation_history`：儲存對話歷史（Response API 格式）

這樣可以在多次函數呼叫間保持狀態。

### 教學法整合

**費曼學習法**：
- 強制模型用簡單語言解釋
- 要求使用類比和例子
- 避免直接使用專業術語

**蘇格拉底式提問**：
- 不直接給答案
- 引導學生思考
- 提出啟發性問題

---

## 🚀 可能的改進方向

1. **PDF 分段處理**：對於超長論文，可以先分析結構，讓使用者選擇要讀哪一段
2. **視覺化**：產生論文結構圖、概念關係圖
3. **筆記功能**：讓使用者儲存重要的問答
4. **多論文比較**：上傳多篇論文，比較異同
5. **匯出功能**：將對話匯出為筆記文件

---

## 📝 作業說明

### 設計理念

這個論文閱讀助手的核心價值在於：

1. **不只是摘要工具**：不是簡單的 TL;DR，而是互動式學習助手
2. **教學導向**：整合費曼學習法和蘇格拉底式提問法
3. **技術展示**：使用最新的 Response API，展示對新技術的掌握
4. **實用性**：真正能幫助學生理解論文

### 與老師範例的差異

老師的「員瑛式思考生成器」：
- 單一功能：情緒轉換
- 簡單輸入輸出
- 固定模式

我的「論文閱讀助手」：
- 多功能：PDF 處理 + 多輪對話
- 複雜互動：上傳、提問、清除
- 教學法整合：費曼 + 蘇格拉底
- 技術升級：Response API

**評分優勢**：
- ✅ 避開 6 分陷阱（不只是改變人設）
- ✅ 實用價值高（真的能用來讀論文）
- ✅ 技術展示（Response API + PDF 處理）
- ✅ 教育意義（整合學習理論）

---

## 🔧 故障排除

### 常見問題

**Q: PDF 上傳失敗？**
- 確認檔案是 PDF 格式
- 檢查檔案是否損壞
- 嘗試用其他 PDF 閱讀器開啟確認

**Q: API 錯誤？**
- 確認 Colab Secrets 中有設定 `OpenAI` 金鑰
- 檢查金鑰是否有效且有餘額
- 查看錯誤訊息中的具體原因

**Q: 回應太慢？**
- 降低 `reasoning.effort` 為 `"low"` 或 `"minimal"`
- 降低 `text.verbosity` 為 `"low"`
- 考慮使用 `gpt-5-mini` 或 `gpt-5-nano`

**Q: Token 超過限制？**
- PDF 太長（超過 100 頁建議分段處理）
- 對話歷史太長（點擊清除對話重新開始）

---

## 📚 參考資源

- [OpenAI Response API 文件](https://platform.openai.com/docs/api-reference/responses)
- [Gradio 文件](https://www.gradio.app/docs)
- [PyPDF2 文件](https://pypdf2.readthedocs.io/)
- [費曼學習法](https://en.wikipedia.org/wiki/Feynman_Technique)
- [蘇格拉底式提問法](https://en.wikipedia.org/wiki/Socratic_method)

---

*🤖 Generated with Claude Code + OpenAI GPT-5 Response API*